In [1]:
import os as os
os.chdir('C:\Users\Falco\Desktop\directory\Predicting-wildlife-reservoirs-of-Flaviviruses')
import geopandas as gpd
from matplotlib import pyplot as plt
import shapely
import pandas as pd
import numpy as np
import rasterio
from rasterio import features

In [2]:
spatial = pd.read_pickle('E:\Flavi\Spatial_all_final.pickle') ## IUCN and BIRD life international shapefiles

In [3]:
spatial = spatial[['binomial','geometry']]

In [4]:
#shp_fn = 'cb_2013_us_county_20m.shp'
rst_fn = 'D:/lvstdt/lvstd.tif'
rst = rasterio.open(rst_fn)
meta = rst.meta.copy()
meta.update(compress='lzw')

## Group 1a

In [2]:
Group1a = pd.read_csv('Outputs/Predictions_R_Group1a2.csv')
Group1a["MatchHost"] =  Group1a.MatchHost.str.replace('[^\x00-\x7F]','')

In [3]:
Group1a.PredictionProbability.quantile(.95)

0.012899255933996247

In [20]:
Group1a.PredictionProbability.quantile(.90)

0.004149662412142538

In [7]:
h1a = Group1a[Group1a.PredictionProbability > Group1a.PredictionProbability.quantile(.95)]
g1ma =spatial[spatial['binomial'].isin(h1a.MatchHost.unique().tolist())]
ones = np.ones(len(g1ma))
g1ma['Value'] = ones
g1ma = g1ma.to_crs(epsg = 4326)
g1ma['geometry'] = g1ma.geometry.buffer(0)
g1ma.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group1a2.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [8]:
g1ma = pd.merge(g1ma, Group1a, left_on='binomial', right_on='MatchHost', how='left')

In [9]:
g1ma.reset_index(0, inplace= True)

In [10]:
for i in range (len(g1ma)):
    if i % 40 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g1ma.binomial_x[i])
    TempDf = g1ma.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group1aRasters2/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0


C:\Users\Falco\Anaconda2\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


40
80


## Group 2

In [4]:
Group2 = pd.read_csv('Outputs/Predictions_R_Group22.csv')
Group2["MatchHost"] =  Group2.MatchHost.str.replace('[^\x00-\x7F]','')

In [5]:
Group2.PredictionProbability.quantile(.95)

0.046211927201670416

In [6]:
Group2.PredictionProbability.quantile(.90)

0.01748294902395392

In [7]:
Group2.PredictionProbability.quantile(.99)

0.3423065584692536

In [16]:

h2 = Group2[Group2.PredictionProbability > Group2.PredictionProbability.quantile(.95)]
g2 =spatial[spatial['binomial'].isin(h2.MatchHost.unique().tolist())]
ones = np.ones(len(g2))
g2.shape

(708, 2)

In [17]:
g2['Value'] = ones
g2 = g2.to_crs(epsg = 4326)
g2['geometry'] = g2.geometry.buffer(0)
g2.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group22.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
g2.reset_index(0, inplace= True)
#g2.head()

In [19]:
for i in range (len(g2)):
    if i % 50 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g2.binomial[i])
    
    TempDf = g2.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group2Rasters2/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700


# Group 2 PCR only

In [5]:
Group2 = pd.read_csv('Outputs/Predictions_R_Group22_PCR_only.csv')
Group2["MatchHost"] =  Group2.MatchHost.str.replace('[^\x00-\x7F]','')
h2 = Group2[Group2.PredictionProbability > Group2.PredictionProbability.quantile(.95)]
g2 =spatial[spatial['binomial'].isin(h2.MatchHost.unique().tolist())]
ones = np.ones(len(g2))
g2.shape

(708, 2)

In [6]:
g2['Value'] = ones
g2 = g2.to_crs(epsg = 4326)
g2['geometry'] = g2.geometry.buffer(0)
g2.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group22_PCR.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
g2.reset_index(0, inplace= True)
#g2.head()

In [8]:
for i in range (len(g2)):
    if i % 50 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g2.binomial[i])
    
    TempDf = g2.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group2Rasters2_PCR/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0


C:\Users\Falco\Anaconda2\lib\site-packages\rasterio\__init__.py:160: FutureWarning: GDAL-style transforms are deprecated and will not be supported in Rasterio 1.0.
  transform = guard_transform(transform)


50
100
150
200
250
300
350
400
450
500
550
600
650
700


# Group 3

In [8]:
Group3 = pd.read_csv('Outputs/Predictions_R_Group3.csv')
Group3["MatchHost"] =  Group3.MatchHost.str.replace('[^\x00-\x7F]','')

In [9]:
Group3.PredictionProbability.quantile(.95)

0.0047199697552986145

In [10]:
Group3.PredictionProbability.quantile(.90)

0.0019789091384486666

In [20]:
Group3 = pd.read_csv('Outputs/Predictions_R_Group3.csv')
Group3["MatchHost"] =  Group3.MatchHost.str.replace('[^\x00-\x7F]','')
h3 = Group3[Group3.PredictionProbability > Group3.PredictionProbability.quantile(.95)]
g3 =spatial[spatial['binomial'].isin(h3.MatchHost.unique().tolist())]
ones = np.ones(len(g3))
g3.shape

(569, 2)

In [21]:
g3['Value'] = ones
g3 = g3.to_crs(epsg = 4326)
g3['geometry'] = g3.geometry.buffer(0)
g3.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group3.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
#g3 = gpd.read_file('Group3.shp')

In [23]:
g3.reset_index(0, inplace= True)

In [24]:
for i in range (len(g3)):
    if i % 50 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g3.binomial[i])
    
    TempDf = g3.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group3Rasters/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0
50
100
150
200
250
300
350
400
450
500
550


# Group 4

In [11]:
Group4 = pd.read_csv('Outputs/Predictions_R_Group4.csv')
Group4["MatchHost"] =  Group4.MatchHost.str.replace('[^\x00-\x7F]','')

In [12]:
Group4.PredictionProbability.quantile(.95)

0.027568383828830076

In [13]:
Group4.PredictionProbability.quantile(.90)

0.012990182356786582

In [25]:
Group4 = pd.read_csv('Outputs/Predictions_R_Group4.csv')
Group4["MatchHost"] =  Group4.MatchHost.str.replace('[^\x00-\x7F]','')
h4 = Group4[Group4.PredictionProbability > Group4.PredictionProbability.quantile(.95)]
g4 =spatial[spatial['binomial'].isin(h4.MatchHost.unique().tolist())]
ones = np.ones(len(g4))
g4.shape

(56, 2)

In [26]:
g4['Value'] = ones
g4 = g4.to_crs(epsg = 4326)
g4['geometry'] = g4.geometry.buffer(0)
g4.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group4.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [27]:
#g4 = gpd.read_file('Group4.shp')

In [28]:
g4.reset_index(0, inplace= True)

In [29]:
g4.head()

,index,binomial,geometry,Value
0,56,Aethalops aequalis,"(POLYGON ((115.8714008336816 2.96083640937286,...",1.0
1,283,Artibeus jamaicensis,(POLYGON ((-61.07291705337929 14.8428384756152...,1.0
2,284,Artibeus lituratus,(POLYGON ((-108.7359999994004 25.4050355925186...,1.0
3,299,Asellia tridens,(POLYGON ((49.24635634786722 30.45191798542675...,1.0
4,564,Carollia brevicauda,POLYGON ((-57.13760000003992 5.835935592553597...,1.0


In [30]:
for i in range (len(g4)):
    if i % 50 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g4.binomial[i])
    
    TempDf = g4.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group4Rasters/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0
50


# Group 5

In [14]:
Group5 = pd.read_csv('Outputs/Predictions_R_Group5.csv')
Group5["MatchHost"] =  Group5.MatchHost.str.replace('[^\x00-\x7F]','')

In [15]:
Group5.PredictionProbability.quantile(.95)

0.009401517882255602

In [16]:
Group5.PredictionProbability.quantile(.90)

0.0035451069961270013

In [11]:
Group5 = pd.read_csv('Outputs/Predictions_R_Group5.csv')
Group5["MatchHost"] =  Group5.MatchHost.str.replace('[^\x00-\x7F]','')
h5 = Group5[Group5.PredictionProbability > Group5.PredictionProbability.quantile(.95)]
g5 =spatial[spatial['binomial'].isin(h5.MatchHost.unique().tolist())]
ones = np.ones(len(g5))

g5['Value'] = ones
g5 = g5.to_crs(epsg = 4326)
g5['geometry'] = g5.geometry.buffer(0)
g5.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group5.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [12]:
#g5 = gpd.read_file('Group5.shp')

In [13]:
g5.reset_index(0, inplace= True)

In [14]:
for i in range (len(g5)):
    if i % 50 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g5.binomial[i])
    
    TempDf = g5.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group5Rasters/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0
50
100
150


# Group 6

In [17]:
Group6 = pd.read_csv('Outputs/Predictions_R_Group6.csv')
Group6["MatchHost"] =  Group6.MatchHost.str.replace('[^\x00-\x7F]','')

In [18]:
Group6.PredictionProbability.quantile(.95)

0.0031019414742902754

In [19]:
Group6.PredictionProbability.quantile(.90)

0.00200298635320534

In [35]:
Group6 = pd.read_csv('Outputs/Predictions_R_Group6.csv')
Group6["MatchHost"] =  Group6.MatchHost.str.replace('[^\x00-\x7F]','')
h6 = Group6[Group6.PredictionProbability > Group6.PredictionProbability.quantile(.95)]
g6 =spatial[spatial['binomial'].isin(h6.MatchHost.unique().tolist())]
ones = np.ones(len(g6))

g6['Value'] = ones
g6 = g6.to_crs(epsg = 4326)
g6['geometry'] = g6.geometry.buffer(0)
g6.to_file(driver='ESRI Shapefile',filename=r'Revision_shapefiles/Group6.shp')

C:\Users\Falco\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [36]:
#g6 = gpd.read_file('Group6.shp')

In [37]:
g6.reset_index(0, inplace= True)

In [38]:
for i in range (len(g6)):
    if i % 50 == 0:
        print i
    #print Generic.binomial[i]
    name = str(g6.binomial[i])
    
    TempDf = g6.iloc[i].to_frame()
    TempDf = TempDf.transpose().reset_index()

    out_fn = 'data\SpeciesRasters/Group6Rasters/'+ name +'.tif'
    with rasterio.open(out_fn, 'w', **meta) as out:
        out_arr = out.read(1)

        # this is where we create a generator of geom, value pairs to use in rasterizing
        shapes = ((geom,value) for geom, value in zip(TempDf.geometry, TempDf.Value))
        burned = features.rasterize(shapes=shapes, fill=1, out=out_arr, transform=out.transform)
        out.write_band(1, burned)

0
50
100
150
200
250
300
350
400
